In [1]:
%pip install google-generativeai

  Using cached google_generativeai-0.8.5-py3-none-any.whl (155 kB)
  Using cached google_api_core-2.24.2-py3-none-any.whl (160 kB)
  Using cached google_ai_generativelanguage-0.6.15-py3-none-any.whl (1.3 MB)
  Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.2/316.2 KB 2.4 MB/s eta 0:00:00a 0:00:01
  Using cached google_api_python_client-2.168.0-py3-none-any.whl (13.3 MB)
  Using cached pydantic-2.11.3-py3-none-any.whl (443 kB)
  Using cached google_auth-2.39.0-py2.py3-none-any.whl (212 kB)
  Using cached protobuf-5.29.4-cp38-abi3-manylinux2014_x86_64.whl (319 kB)
  Using cached proto_plus-1.26.1-py3-none-any.whl (50 kB)
  Using cached cachetools-5.5.2-py3-none-any.whl (10 kB)
  Using cached rsa-4.9.1-py3-none-any.whl (34 kB)
  Using cached pyasn1_modules-0.4.2-py3-none-any.whl (181 kB)
  Using cached googleapis_common_protos-1.70.0-py3-none-any.whl (294 kB)
  Using cached requests-2.32.3-py3-none-any.whl (64 kB)
  Using cached uritempl

In [ ]:
import google.generativeai as genai
from dotenv import load_dotenv
import os

load_dotenv()  

API_KEY = os.getenv("API_KEY")

genai.configure(api_key=API_KEY)

model = genai.GenerativeModel('gemini-2.0-flash')

response = model.generate_content(
    "Write a dramatic short story about a dragon."
)


print(response.text)


/home/shtlp_0012/codes/Project_RAG/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The wind clawed at Ignis, ripping at the obsidian scales on his chest like a ravenous beast. Below, the jagged peaks of the Dragon's Tooth Mountains plunged into a swirling abyss of storm clouds. He hadn't flown this low in centuries, not since... not since he'd lost everything.

The memory, a burning ember in his heart, flickered to life. The lush valley, teeming with life, the soft laughter of his clutchlings, the warmth of his mate's scales pressed against his. All gone. Scourged by the iron teeth and fire of the humans.

A guttural rumble clawed its way from his throat. He had vowed vengeance then, a promise carved into the very core of his being. But centuries had dulled the rage, leaving behind only a cold, gnawing ache.

He spotted it then, a faint flicker of light nestled in the crook of the mountain, like a single ember refusing to be extinguished. A village. Human village.

His claws tightened on the winds, pulling him towards the light. The scent of smoke, of fear, filled hi

In [2]:
%pip install sentence-transformers faiss-cpu

  Using cached sentence_transformers-4.1.0-py3-none-any.whl (345 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 7.3 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 9.8 MB/s eta 0:00:00:00:0100:01
  Using cached scikit_learn-1.6.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.5 MB)
  Using cached scipy-1.15.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (37.6 MB)
  Using cached transformers-4.51.3-py3-none-any.whl (10.4 MB)
  Using cached torch-2.7.0-cp310-cp310-manylinux_2_28_x86_64.whl (865.2 MB)
  Using cached huggingface_hub-0.30.2-py3-none-any.whl (481 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 9.3 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 751.2/751.2 KB 8.2 MB/s eta 0:00:00a 0:00:01
  Using cached filelock-3.18.0-py3-none-any.whl (16 kB)
  Using cached fsspec-2025.3.2-py3-none-any.whl (194 kB)
  Using cached nvidia_cusparse_cu12-12.5.4.2-py3-none-m

In [64]:
with open("Sherlock_Holmes.txt", "r") as f:
    text = f.read()

def split_text(text, chunk_size=500, overlap=50):
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start = end - overlap  # small overlap to avoid breaking context
    return chunks

# Example
chunks = split_text(text)


In [65]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# 1. Create Embeddings
embed_model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = embed_model.encode(chunks, show_progress_bar=True)

# 2. Ensure float32 for FAISS
embeddings = np.array(embeddings).astype(np.float32)

# 3. Build FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

print("✅ FAISS index built successfully!")


Batches: 100%|██████████| 41/41 [00:20<00:00,  2.00it/s]


✅ FAISS index built successfully!


In [73]:
def search_index(query, top_k=5):
    query_embedding = embed_model.encode([query])
    distances, indices = index.search(np.array(query_embedding), top_k)
    results = [chunks[i] for i in indices[0]]
    return results

# Example
relevant_chunks = search_index("Holmes", top_k=3)
print(relevant_chunks)


['g his\nface. And here he comes, if I am not mistaken, to resolve all our\ndoubts.”\n\nAs he spoke there was the sharp sound of horses’ hoofs and grating\nwheels against the curb, followed by a sharp pull at the bell. Holmes\nwhistled.\n\n“A pair, by the sound,” said he. “Yes,” he continued, glancing out of\nthe window. “A nice little brougham and a pair of beauties. A hundred\nand fifty guineas apiece. There’s money in this case, Watson, if there\nis nothing else.”\n\n“I think that I had better go, Holmes.”\n', 'und, and I could see in the gas-light\nthat every vestige of colour had been driven from his face.\n\n“Who are you, then? What do you want?” he asked in a quavering voice.\n\n“You will excuse me,” said Holmes blandly, “but I could not help\noverhearing the questions which you put to the salesman just now. I\nthink that I could be of assistance to you.”\n\n“You? Who are you? How could you know anything of the matter?”\n\n“My name is Sherlock Holmes. It is my business to know w

In [74]:
full_context = "\n".join(relevant_chunks)
prompt = f"""
Based on the following document content, answer the user's question:\n\n{full_context}\n\nQuestion: What can we learn about Holmes’s personality from his investigation methods?. \nAnswer:
"""

response = model.generate_content(prompt)
print(response.text)


DeadlineExceeded: 504 Deadline Exceeded

In [ ]:
%pip install weaviate-client

In [66]:
import weaviate

client = weaviate.Client(url="http://localhost:8080")

# Define the schema
class_obj = {
    "class": "HolmesChunk",  # You can name it whatever you want
    "vectorizer": "none",  # Important! We provide our own vectors
    "properties": [
        {
            "name": "text",
            "dataType": ["text"]
        }
    ]
}

# Create the schema
client.schema.create_class(class_obj)

print("✅ Schema created successfully!")


✅ Schema created successfully!


In [67]:
import numpy as np

# Make sure embeddings are float32
embeddings = np.array(embeddings).astype(np.float32)

for chunk, embedding in zip(chunks, embeddings):
    client.data_object.create(
        data_object={"text": chunk},
        class_name="HolmesChunk",
        vector=embedding.tolist()
    )

print("✅ All data inserted into Weaviate!")


✅ All data inserted into Weaviate!


In [69]:
# 1. User asks a question
query = "Who was Sherlock Holmes?"

# 2. Embed the query
query_embedding = embed_model.encode([query])
query_embedding = np.array(query_embedding).astype(np.float32)

# 3. Search Weaviate for similar chunks
response = client.query.get(
    "HolmesChunk",
    ["text", "_additional { vector }"]
).with_near_vector({
    "vector": query_embedding[0]
}).with_limit(5).do()

# 4. Print the best matches
for i, result in enumerate(response["data"]["Get"]["HolmesChunk"]):
    print(f"Result {i+1}:\n{result['text']}\n")



Result 1:
und, and I could see in the gas-light
that every vestige of colour had been driven from his face.

“Who are you, then? What do you want?” he asked in a quavering voice.

“You will excuse me,” said Holmes blandly, “but I could not help
overhearing the questions which you put to the salesman just now. I
think that I could be of assistance to you.”

“You? Who are you? How could you know anything of the matter?”

“My name is Sherlock Holmes. It is my business to know what other
people don’t know.”



Result 2:
t. He used to make merry over the cleverness of women, but I
have not heard him do it of late. And when he speaks of Irene Adler, or
when he refers to her photograph, it is always under the honourable
title of _the_ woman.




II. THE RED-HEADED LEAGUE


 I had called upon my friend, Mr. Sherlock Holmes, one day in the
 autumn of last year and found him in deep conversation with a very
 stout, florid-faced, elderly gentleman with fiery red hair. With an
 apology for my intr

In [55]:
client.schema.delete_all()
